In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../../src/')
import data_utils
import functools
import imageio
import cv2
import os
from tqdm.auto import tqdm

### Load up the data & put together!

In [2]:
base_data_path = '../../data/'

#### nabirds

In [3]:
# Just read the data
nabirds_path = '../../data/nabirds/'
nabirds_labels = ['images.txt', 'bounding_boxes.txt']
nabirds_columns = {
    'images.txt' : ['id', 'fname'],
    'bounding_boxes.txt' : ['id', 'x_bb', 'y_bb', 'width_bb', 'height_bb'],
}
nabirds_dfs = [pd.read_csv(nabirds_path + nabirds_label, delimiter=' ', names=nabirds_columns[nabirds_label]) for nabirds_label in nabirds_labels]
nabirds_df = functools.reduce(lambda x, y: pd.merge(x, y, on='id'), nabirds_dfs)
nabirds_df['source'] = 'nabirds'
nabirds_df['multi'] = False
nabirds_df['load_path'] = nabirds_df['fname'].apply(lambda x: base_data_path + 'nabirds/images/'+x)
nabirds_df.drop(['fname'], axis=1, inplace=True)
nabirds_df.head(5)

,id,x_bb,y_bb,width_bb,height_bb,source,multi,load_path
0,0000139e-21dc-4d0c-bfe1-4cae3c85c829,83,59,128,228,nabirds,False,../../data/nabirds/images/0817/0000139e21dc4d0...
1,0000d9fc-4e02-4c06-a0af-a55cfb16b12b,328,88,163,298,nabirds,False,../../data/nabirds/images/0860/0000d9fc4e024c0...
2,00019306-9d83-4334-b255-a447742edce3,174,367,219,378,nabirds,False,../../data/nabirds/images/0900/000193069d83433...
3,0001afd4-99a1-4a67-b940-d419413e23b3,307,179,492,224,nabirds,False,../../data/nabirds/images/0645/0001afd499a14a6...
4,000332b8-997c-4540-9647-2f0a8495aecf,395,139,262,390,nabirds,False,../../data/nabirds/images/0929/000332b8997c454...


In [4]:
# Just read the data
cub_path = '../../data/CUB_200_2011/'
cub_columns = {
    'images.txt' : ['id', 'fname'],
    'bounding_boxes.txt' : ['id', 'x_bb', 'y_bb', 'width_bb', 'height_bb'],
    '/parts/parts_locs.txt' : ['id', 'part_id', 'x_part', 'y_part', 'visible'],
}
cub_labels = ['images.txt', 'bounding_boxes.txt']
cub_img_dfs = [pd.read_csv(cub_path + img_label, delimiter=' ', names=cub_columns[img_label]) for img_label in cub_labels]
cub_df = functools.reduce(lambda x, y: pd.merge(x, y, on='id'), cub_img_dfs)
cub_df['source'] = 'cub'
cub_df['multi'] = False
cub_df['load_path'] = cub_df['fname'].apply(lambda x: base_data_path + 'CUB_200_2011/images/'+x)
cub_df.drop(['fname',], axis=1, inplace=True)
cub_df.head(5)

,id,x_bb,y_bb,width_bb,height_bb,source,multi,load_path
0,1,60.0,27.0,325.0,304.0,cub,False,../../data/CUB_200_2011/images/001.Black_foote...
1,2,139.0,30.0,153.0,264.0,cub,False,../../data/CUB_200_2011/images/001.Black_foote...
2,3,14.0,112.0,388.0,186.0,cub,False,../../data/CUB_200_2011/images/001.Black_foote...
3,4,112.0,90.0,255.0,242.0,cub,False,../../data/CUB_200_2011/images/001.Black_foote...
4,5,70.0,50.0,134.0,303.0,cub,False,../../data/CUB_200_2011/images/001.Black_foote...


In [5]:
obd_path = '../../data/oriental_bird_database/'
obd_df = pd.read_csv(obd_path + 'obd_updated_labels.csv')
obd_df.drop(['Unnamed: 0', 'url'], axis=1, inplace=True)
obd_df = obd_df[obd_df['have_image']]
obd_df.drop('have_image', axis=1, inplace=True)
obd_df['source'] = 'obd'
obd_df['load_path'] = obd_df.apply(lambda x: base_data_path + 'oriental_bird_database/images/' + str(x['bird_id']) + '_' + str(x['image_id']) + '.jpg', axis=1)
obd_df.drop(['bird_id', 'image_id'], axis=1, inplace=True)
obd_df['id'] = ""
obd_df.head(5)

,x_bb,y_bb,width_bb,height_bb,multi,source,load_path,id
0,126.020386,128.291960,282.106540,143.515990,False,obd,../../data/oriental_bird_database/images/208_7...,
1,135.428220,139.247330,269.109220,188.306410,False,obd,../../data/oriental_bird_database/images/208_7...,
2,56.632763,159.360780,510.863160,184.702700,False,obd,../../data/oriental_bird_database/images/208_5...,
3,254.112520,170.061610,311.460800,173.392610,False,obd,../../data/oriental_bird_database/images/218_1...,
4,90.416702,96.904793,296.336365,120.375496,True,obd,../../data/oriental_bird_database/images/218_1...,


In [6]:
flikr_path = '../../data/flikr/'
flikr_df = pd.read_csv(flikr_path + 'labels.csv')
flikr_df['width_bb'] = flikr_df['width']
flikr_df['height_bb'] = flikr_df['height']
flikr_df.drop(['Unnamed: 0', 'width', 'height'], axis=1, inplace=True)
flikr_df['source'] = 'flikr'
flikr_df['load_path'] = flikr_df['fname'].apply(lambda x: base_data_path + 'flikr/images/' + x)
flikr_df.drop(['fname'], axis=1, inplace=True)
flikr_df['id'] = ""
flikr_df.head(5)

,x_bb,y_bb,multi,width_bb,height_bb,source,load_path,id
0,147.844284,232.497345,False,741.062378,492.207672,flikr,../../data/flikr/images/48767409097.jpg,
1,1369.728760,244.971420,False,526.283813,690.795776,flikr,../../data/flikr/images/48219852712.jpg,
2,159.751404,51.135323,False,1710.590576,1071.756104,flikr,../../data/flikr/images/46864697344.jpg,
3,219.261368,35.987675,False,1479.295532,1263.510864,flikr,../../data/flikr/images/42603475312.jpg,
4,586.860229,119.171722,False,832.564819,1042.072632,flikr,../../data/flikr/images/48034176297.jpg,


### Put the data together

In [7]:
full_df = pd.concat([nabirds_df, cub_df, obd_df, flikr_df], sort=True).reset_index(drop=True)

In [8]:
full_df.sample(5)

,height_bb,id,load_path,multi,source,width_bb,x_bb,y_bb
178529,564.151123,,../../data/flikr/images/48029341316.jpg,False,flikr,747.034729,880.961487,402.533020
58436,100.000000,9875,../../data/CUB_200_2011/images/168.Kentucky_Wa...,False,cub,182.000000,272.000000,149.000000
122193,419.168200,,../../data/oriental_bird_database/images/2422_...,False,obd,227.638080,146.934000,65.520310
149363,284.697750,,../../data/oriental_bird_database/images/1518_...,False,obd,373.877200,167.638140,103.339386
37955,295.000000,c78269d3-8cee-4846-ac84-cc8b23087285,../../data/nabirds/images/0775/c78269d38cee484...,False,nabirds,243.000000,346.000000,241.000000


In [9]:
def load_and_preview(row):
    img = imageio.imread(row['load_path'])
    data_utils.preview_img(img, row['x_bb'], row['y_bb'], row['width_bb'], row['height_bb'])

In [10]:
full_df.shape

(198267, 8)

#### subset to bounding boxes & images

In [11]:
full_df = full_df[full_df['width_bb'] > 0]
full_df.shape

(196233, 8)

### Throw away images that are too low resolution or bad aspect ratio

In [12]:
target_resolution = 256

In [13]:
full_df['bird_resolution'] = full_df.apply(lambda x: max(x['width_bb'], x['height_bb'])**2, axis=1)

In [14]:
full_df['good_resolution'] = full_df['bird_resolution'].apply(lambda x: x > target_resolution**2)
(~full_df['good_resolution']).sum()

25375

In [15]:
np.random.seed(42)
full_df = full_df[(full_df['good_resolution'])].sample(frac=1).reset_index(drop=True).reset_index()
full_df.rename({'index' : 'img_id'}, axis=1, inplace=True)

### Denoise and save the images!

In [16]:
full_df.head(5)

,img_id,height_bb,id,load_path,multi,source,width_bb,x_bb,y_bb,bird_resolution,good_resolution
0,0,574.000000,4185bdac-1020-4ec6-920b-761bd811d7de,../../data/nabirds/images/0980/4185bdac10204ec...,False,nabirds,256.000000,251.000000,38.000000,3.294760e+05,True
1,1,299.752870,,../../data/oriental_bird_database/images/298_4...,False,obd,452.408700,22.924822,126.878980,2.046736e+05,True
2,2,253.836685,,../../data/oriental_bird_database/images/2409_...,True,obd,298.087189,11.979636,10.181627,8.885597e+04,True
3,3,1249.867920,,../../data/flikr/images/48245398241.jpg,False,flikr,446.096130,229.657455,0.000000,1.562170e+06,True
4,4,378.722530,,../../data/oriental_bird_database/images/737_7...,False,obd,309.916800,139.623490,75.924990,1.434308e+05,True


In [17]:
save_dir = '../../data/first_pass/'
image_dir = save_dir + 'images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(image_dir):
    os.mkdir(image_dir)

In [19]:
full_df.to_csv(save_dir + 'labels.csv', index=False)

In [ ]:
tqdm.pandas()

In [ ]:
def show_big(img):
    fig = plt.gcf()
    fig.set_size_inches(8,8)
    plt.imshow(img)
    plt.show()

In [ ]:
target_row = full_df.sample(1).iloc[0]
img = data_utils.preprocess_img(target_row, target_resolution, target_resolution, preview=False)
img = cv2.fastNlMeansDenoisingColored(img, None, 3, 3, 5, 7)
show_big(img)

In [ ]:
failed = full_df.progress_apply(data_utils.save_img, save_path=image_dir, target_width=target_resolution, target_height=target_resolution, axis=1)